In [27]:
import pandas as pd 
import numpy as np 
import os 
import requests
from pprint import pprint
import matplotlib.pyplot as plt
import seaborn as sns
import glob

In [28]:
import requests
import pprint

teams = [
    
    # "BOS",  # Boston Bruins
    # "ANA",  # Anaheim Ducks
    # "BUF",  # Buffalo Sabres
    # "CAR",  # Carolina Hurricanes
    # "CBJ",  # Columbus Blue Jackets
    # "CGY",  # Calgary Flames
    # "CHI",  # Chicago Blackhawks
    # "COL",  # Colorado Avalanche
    # "DAL",  # Dallas Stars
    # "DET",  # Detroit Red Wings
    #                                 "EDM",  # Edmonton Oilers
    # "FLA",  # Florida Panthers
    # "LAK",  # Los Angeles Kings
    # "MIN",  # Minnesota Wild
    # "MTL",  # Montreal Canadiens
    # "NJD",  # New Jersey Devils
    # "NSH",  # Nashville Predators
    # "NYI",  # New York Islanders
    #                                 "NYR",  # New York Rangers
    # "OTT",  # Ottawa Senators
    # "PHI",  # Philadelphia Flyers
    # "PIT",  # Pittsburgh Penguins
    # "SEA",  # Seattle Kraken
    # "SJS",  # San Jose Sharks
    # "STL",  # St. Louis Blues
    # "TBL",  # Tampa Bay Lightning
    # "TOR",  # Toronto Maple Leafs
    # "UTA",  # Utah Hockey Club
                                   "VAN",  # Vancouver Canucks
    # "VGK",  # Vegas Golden Knights
    # "WPG",  # Winnipeg Jets
    # "WSH",   # Washington Capitals
    # "ARI",   # Arizona Coyotes
    # "ATL",   # Atlanta Thrashers 
    # "HFD",   # Hartford
    # "PHX",   # Pheonix
    # "QUE",   # Quebec
    # "WIN",   # Winnepeg Jets - Old
    ]

seasons = [

    "19701971", "19711972", "19721973", "19731974", "19741975",
    "19751976", "19761977", "19771978", "19781979", "19791980",
    "19801981", "19811982", "19821983", "19831984", "19841985",
    "19851986", "19861987", "19871988", "19881989", "19891990",
    "19901991", "19911992", "19921993", "19931994", "19941995",
    "19951996", "19961997", "19971998", "19981999", "19992000",
    "20002001", "20012002", "20022003", "20032004", "20042005",
    "20052006", "20062007", "20072008", "20082009", "20092010",
    "20102011", "20112012", "20122013", "20132014", "20142015",
    "20152016", "20162017", "20172018", "20182019", "20192020",
    "20202021", "20212022", "20222023", "20232024", "20242025"
]

# Function to get player data from the NHL API
def get_player_data(team, season):
    url = f"https://api-web.nhle.com/v1/roster/{team}/{season}"
    response = requests.get(url)

    # Check if the response is valid JSON
    try:
        return response.json()
    except requests.JSONDecodeError:
        print(f"Error decoding JSON for team {team} in season {season}")
        return {}

# Initialize a dictionary to store player IDs
player_ids = {}

# Loop through each season in the seasons list
for season in seasons:
    player_ids[season] = {}

    # Loop through each team
    for team in teams:
        try:
            # Get the player data for the team and season
            player_data = get_player_data(team, season)
        
            # Initialize dictionaries for defencemen, forwards, and goalies
            player_ids[season][team] = {
                'defencemen': [],
                #'forwards': [],
                #'goalies': []
            }
        
            # Extract player IDs from defencemen
            if 'defensemen' in player_data:
                for player in player_data['defensemen']:
                    player_id = player['id']
                    player_ids[season][team]['defencemen'].append(player_id)
        
            # # Extract player IDs from forwards
            # if 'forwards' in player_data:
            #     for player in player_data['forwards']:
            #         player_id = player['id']
            #         player_ids[season][team]['forwards'].append(player_id)
        
            # # Extract player IDs from goalies
            # if 'goalies' in player_data:
            #     for player in player_data['goalies']:
            #         player_id = player['id']
            #         player_ids[season][team]['goalies'].append(player_id)
    
        except KeyError:
            print(f"Team {team} not found for season {season}. Skipping...")
            continue

#Print the player IDs
pprint.pprint(player_ids)

Error decoding JSON for team VAN in season 20042005
{'19701971': {'VAN': {'defencemen': [8444966,
                                     8445100,
                                     8446434,
                                     8446698,
                                     8448178,
                                     8448240,
                                     8448259,
                                     8451169,
                                     8449055,
                                     8451853,
                                     8449473,
                                     8449667]}},
 '19711972': {'VAN': {'defencemen': [8445104,
                                     8446434,
                                     8447420,
                                     8448400,
                                     8448178,
                                     8448240,
                                     8451169,
                                     8451853,
                         

In [29]:
# Function to get game logs for a specific season
def get_game_log(player_id, season):
    url = f"https://api-web.nhle.com/v1/player/{player_id}/game-log/{season}/2"
    response = requests.get(url)
    return response.json()

# Check if the response is valid JSON
    try:
        return response.json()
    except requests.JSONDecodeError:
        print(f"Error decoding JSON for player {player_id} in season {season}")
        return {}

In [30]:
import pandas as pd

# Function to aggregate game logs for a specific player_id
def aggregate_game_logs(game_logs):
    aggregated_data = {}
    
    for game in game_logs:
        for key, value in game.items():
            if key not in aggregated_data:
                aggregated_data[key] = value
            else:
                # If the key already exists, aggregate the data (e.g., sum the values)
                if isinstance(value, (int, float)):
                    aggregated_data[key] += value
                elif isinstance(value, list):
                    aggregated_data[key].extend(value)
                # Handle other data types as needed
    
    return aggregated_data



# Initialize a dictionary to store DataFrames for each player_id
player_dfs = {}

# Loop through the selected player_ids
for season, teams in player_ids.items():
    for team, players in teams.items():
        for player_type, player_ids in players.items():
            for player_id in player_ids:
                game_log = get_game_log(player_id, season)
                
                if game_log and 'gameLog' in game_log:
                    game_log_data = game_log['gameLog']

                    seasons_list = []
                    
                    for stat in game_log_data:
                        stats_info = {
                            'gameDate': stat.get('gameDate'),
                            'homeRoadFlag': stat.get('homeRoadFlag'),    
                            'goals': stat.get('goals'),    
                            'assists': stat.get('assists'),    
                            'points': stat.get('points'),    
                            'plusMinus': stat.get('plusMinus'),    
                            'powerPlayGoals': stat.get('powerPlayGoals'),    
                            'powerPlayPoints': stat.get('powerPlayPoints'),    
                            'gameWinningGoals': stat.get('gameWinningGoals'),    
                            'otGoals': stat.get('otGoals'),    
                            'shots': stat.get('shots'),    
                            'shifts': stat.get('shifts'),        
                            'pim': stat.get('pim')
                        }
                        seasons_list.append(stats_info)
                    
                    # Create df from the flattened list of data
                    player_df = pd.DataFrame(seasons_list)

                    #  # Set gameDate as the index
                    # player_df.set_index('gameDate', inplace=True)

                    # Add the DataFrame to the player_dfs dictionary
                    player_dfs[player_id] = player_df

  
                   # Save the DataFrame to a CSV file
                    csv_filename = f'player_{player_id}.csv'
                    player_df.to_csv(csv_filename)
                    print(f"DataFrame for player_id {player_id} saved to {csv_filename}")
                else:
                    print(f"No valid data found for player_id {player_id} in season {season}")




DataFrame for player_id 8444966 saved to player_8444966.csv
DataFrame for player_id 8445100 saved to player_8445100.csv
DataFrame for player_id 8446434 saved to player_8446434.csv
DataFrame for player_id 8446698 saved to player_8446698.csv
DataFrame for player_id 8448178 saved to player_8448178.csv
DataFrame for player_id 8448240 saved to player_8448240.csv
DataFrame for player_id 8448259 saved to player_8448259.csv
DataFrame for player_id 8451169 saved to player_8451169.csv
DataFrame for player_id 8449055 saved to player_8449055.csv
DataFrame for player_id 8451853 saved to player_8451853.csv
DataFrame for player_id 8449473 saved to player_8449473.csv
DataFrame for player_id 8449667 saved to player_8449667.csv
DataFrame for player_id 8445104 saved to player_8445104.csv
DataFrame for player_id 8446434 saved to player_8446434.csv
DataFrame for player_id 8447420 saved to player_8447420.csv
DataFrame for player_id 8448400 saved to player_8448400.csv
DataFrame for player_id 8448178 saved to

In [31]:
# Function to load all CSV files into a dictionary of DataFrames
def load_all_csv_files(directory):
    csv_files = glob.glob(os.path.join(directory, 'player_*.csv'))
    dataframes = {}
    
    for csv_file in csv_files:
        player_id = os.path.basename(csv_file).split('_')[1].split('.')[0]
        df = pd.read_csv(csv_file)
        dataframes[player_id] = df
    
    return dataframes

# Specify the directory where the CSV files are saved
directory = '.'  # Current directory

# Load all CSV files into a dictionary of DataFrames
player_dfs = load_all_csv_files(directory)

In [32]:
# Function to create a new DataFrame grouped by homeRoadFlag
def create_grouped_df(df):
    grouped_df = df.groupby('homeRoadFlag').agg({
        'gameDate': 'count',
        'goals': 'sum',
        'assists': 'sum',
        'points': 'sum',
        'plusMinus': 'sum',
        'powerPlayGoals': 'sum',
        'powerPlayPoints': 'sum',
        'gameWinningGoals': 'sum',
        'otGoals': 'sum',
        'shots': 'sum',
        'shifts': 'sum',
        'pim': 'sum',
        
    }).rename(columns={'gameDate': 'gameCount'})
    # Sum the grouped statistics to create a single row DataFrame
    single_row_df = grouped_df.sum().to_frame().T
    
    return single_row_df

In [33]:
# Loop through each DataFrame and create a grouped DataFrame
grouped_dfs = {}
for player_id, df in player_dfs.items():
    try:
        # Check if 'gameDate' column has values
        if df['gameDate'].empty:
            print(f"No gameDate values found for player_id {player_id}. Skipping...")
            continue

        # Check if 'homeRoadFlag' column exists
        if 'homeRoadFlag' in df.columns:
            grouped_df = create_grouped_df(df)
            grouped_dfs[player_id] = grouped_df
        else:
            print(f"'homeRoadFlag' column not found for player_id {player_id}. Skipping...")
    except Exception as e:
        print(f"An error occurred for player_id {player_id}: {e}")

An error occurred for player_id 8446316: 'gameDate'
An error occurred for player_id 8480828: 'gameDate'
An error occurred for player_id 8482187: 'gameDate'
An error occurred for player_id 8478846: 'gameDate'
An error occurred for player_id 8477433: 'gameDate'
An error occurred for player_id 8481614: 'gameDate'
An error occurred for player_id 8482641: 'gameDate'
An error occurred for player_id 8478062: 'gameDate'
An error occurred for player_id 8476852: 'gameDate'
An error occurred for player_id 8481570: 'gameDate'
An error occurred for player_id 8468853: 'gameDate'
An error occurred for player_id 8445080: 'gameDate'
An error occurred for player_id 8479341: 'gameDate'
An error occurred for player_id 8479416: 'gameDate'
An error occurred for player_id 8483425: 'gameDate'
An error occurred for player_id 8445298: 'gameDate'
An error occurred for player_id 8482107: 'gameDate'
An error occurred for player_id 8482072: 'gameDate'
An error occurred for player_id 8482138: 'gameDate'
An error occ

In [34]:
# Concatenate all DataFrames into a master DataFrame
master_df = pd.concat(grouped_dfs.values(), keys=grouped_dfs.keys(), names=['player_id', 'index'])

# Reset the index to have 'player_id' as a column
master_df.reset_index(level='index', drop=True, inplace=True)
master_df.to_csv('defence-04.csv')

In [36]:
defence_01 = pd.read_csv('/Users/blairjdaniel/lighthouse/lighthouse/NHL/defence-01.csv')
defence_02 = pd.read_csv('/Users/blairjdaniel/lighthouse/lighthouse/NHL/defence-02.csv')
defence_03 = pd.read_csv('/Users/blairjdaniel/lighthouse/lighthouse/NHL/defence-03.csv')
defence_04 = pd.read_csv('/Users/blairjdaniel/lighthouse/lighthouse/NHL/defence-04.csv')

In [37]:
# Concatenate all DataFrames into one
all_defence = pd.concat([defence_01, defence_02, defence_03, defence_04], ignore_index=True)
# Save the merged DataFrame to a CSV file
all_defence.to_csv('Vancouver.csv', index=False)
print("Merged DataFrame saved to all_defence.csv")

Merged DataFrame saved to all_defence.csv
